<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/dave-updates/code/dave_feature_representations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# !pip install spacy --upgrade
!python -m spacy download en_core_web_sm
# !pip install spacy_readability

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 5.1 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047105 sha256=e0df8473f82bb50d867478ae8a3a213c3c0eef67b2e7981746785f81e3279de5
  Stored in directory: /root/.cache/pip/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.0
    Uninstalling en-core-web-sm-3.4.0:
      Successfully uninstalled en-core-web-sm-3.4.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [13]:
import pandas as pd
import numpy as np
import spacy
from collections import Counter
from spacy_readability import Readability

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [14]:
nlp = spacy.load("en_core_web_sm")
# nlp.disable_pipes(['tok2vec', 'parser', 'attribute_ruler','ner'])
# nlp.add_pipe('sentencizer')
read = Readability()
# nlp.add_pipe('spacy_readability', last=True)
url = 'https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/dale_chall.txt'
df = pd.read_csv(url,header=None, names=['word'])
dc_set = set(df['word'])



In [ ]:
!pip install spacy_readability

In [10]:
def D_ChallEvaluator(x,distinct=False):
  """ 
    x - str; the document to evaluate against Dale-Chall (D-C)
    distinct - boolean to determine whether each matching word counts once (True)
    or count each instance of a matching word (ie if the same dc word 
    appears twice or more)

    Returns  float representing the ratio of D-C words to total length of input 
    string
  """
  

  doc = nlp(x)
  
  # obtain the matched words between input str and the D-C set
  lemma_words = [token.lemma_ for token in doc]
  word_lst = set(lemma_words).intersection(dc_set)

  if distinct:
    # if we are interested in distinct words, just get the length of the set
    word_count = len(word_lst)

  else:  
    # if we look at instances, create a counter of matching D-C words and store
    # their frequencies.
    cntr = Counter([word for word in lemma_words if word in word_lst])
    
    # sum up all of the counts across the matching words
    word_count = np.sum([tup[1] for tup in cntr.items()])
  
  return word_count / len(x.split(' '))







In [11]:
train_url = 'https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/WikiLarge_Train.csv'
train_df = pd.read_csv(train_url)

In [24]:

train_sample = train_df.sample(5000).copy()


# Call the custom built DChall Evaluator Function
train_sample["DC_Ratio"] = train_sample['original_text'].apply(lambda x: D_ChallEvaluator(x,False))

# This attempts to use the dale_chall algorithm built into the spacy readability module.
train_sample['DC_Grade_level'] = train_sample['original_text'].apply(lambda x: nlp(x)._.dale_chall)

In [26]:
train_sample = train_sample[['original_text','DC_Ratio','DC_Grade_level','label']]